In [1]:
import os, sys
sys.path.append("..")

In [2]:
from config.load_config import load_yaml_config, to_dict

In [3]:
import torch

### Generate dataset

In [4]:
from data.synthetic.SyntheticMeshPopulation import SyntheticMeshPopulation
from data.SyntheticDataModules import SyntheticMeshesDataset
from torch.utils.data import DataLoader
from utils.helpers import get_datamodule

In [5]:
config = load_yaml_config("../config_files/config_folded_c_and_s.yaml")

In [6]:
dm = get_datamodule(config)

Retrieving synthetic population from cached file.


___

### `Encoder3DMesh`

In [7]:
from models.Model import Encoder3DMesh

In [8]:
from utils.helpers import get_coma_args

In [9]:
coma_args = get_coma_args(config, dm)
coma_args["phase_input"] = False

In [10]:
encoder_args = [
    "num_features", 
    "n_layers", 
    "n_nodes",
    "num_conv_filters_enc", 
    "polygon_order", 
    "latent_dim_content", 
    "is_variational", 
    "phase_input", 
    "downsample_matrices",
    "adjacency_matrices",
    "activation_layers"
]

decoder_args = [
    "num_features", 
    "n_layers", 
    "n_nodes",
    "num_conv_filters_dec", 
    "polygon_order", 
    "latent_dim_content", 
    "is_variational", 
    "phase_input", 
    "upsample_matrices",
    "adjacency_matrices",
    "activation_layers" 
]

enc_config = {k: v for k,v in coma_args.items() if k in encoder_args}
dec_config = {k: v for k,v in coma_args.items() if k in decoder_args}

In [11]:
# encoder_model = Encoder3DMesh(**enc_config)

In [12]:
# decoder_model = Decoder3DMesh(**dec_config)

In [13]:
datapoint = next(iter(dm.train_dataloader()))
# x = datapoint[0][:,0,:,:]
# z = encoder_model(x[0])
# z

___

### `EncoderTemporalSequence`

In [14]:
from models.Model4D import EncoderTemporalSequence

In [15]:
cine_encoder = EncoderTemporalSequence(enc_config, z_aggr_function="DFT", n_timeframes=20)

In [16]:
s_t = datapoint[0]

In [20]:
cine_encoder(s_t)[0].shape

torch.Size([16, 4])

___

### `ContentDecoder`

In [18]:
from models.Model import ContentDecoder

ImportError: cannot import name 'ContentDecoder' from 'models.Model' (/mnt/HDD/repos/CardiacMotionRL/tests/../models/Model.py)

___

### `StyleDecoder`

In [ ]:
from models.Model import StyleDecoder

___

### `Coma4D_C_and_S`

In [ ]:
from models.Model import Coma4D_C_and_S